In [79]:
from astropy.cosmology import Planck13 as cosmo
from astropy.constants import G
import numpy as np

In [83]:
((3 * (cosmo.H(0)**2)) / (8 * np.pi * G)).to(u.solMass * u.Mpc**-3)

<Quantity 127430827576.2064 solMass / Mpc3>

In [58]:
Mvir = 2.016463 * 10**10 * u.solMass
Mvir

<Quantity 20164630000.0 solMass>

In [59]:
Rvir = 0.04425956 * u.Mpc
Rvir

<Quantity 0.04425956 Mpc>

In [77]:
(((3 * Mvir) / (4 * np.pi * Rvir**3)) / 200) #.decompose()

<Quantity 277619761116.8498 solMass / Mpc3>

In [84]:
((3 * 2.016463) / (4 * np.pi * 1.246 * 10**11))**(1./3) * 200

0.03138276448652702

In [1]:
random = False

import pandas as pd
import numpy as np
import itertools as it
from scipy.spatial.distance import cdist

from hightolowz import distance


print "Reading galaxy data..."

gals = pd.read_csv('data/henriques2015a_z3p10_sfr.csv', skiprows=104, skipfooter=1, engine='python')

n = 100
selection_str = 'mstar9'
redshift_str = '3p95'

print "Filling in NaN values..."
gals.ix[np.isnan(gals['z0_haloId']), 'z0_haloId'] = -1
gals.ix[np.isnan(gals['z0_centralId']), 'z0_centralId'] = -1
gals.ix[np.isnan(gals['z0_central_mcrit200']), 'z0_central_mcrit200'] = 0


L = 480.279

if random:
    print "Initialising random regions..."
    N = 30000
    coods = pd.DataFrame(np.random.rand(N,3) * L, columns=['zn_x','zn_y','zn_z'])
    location_str = 'random'
else:
    print "Copying galaxy coordinates..."
    coods = gals[['zn_x','zn_y','zn_z']].copy()
    location_str = 'gals'

dimensions = np.array([L, L, L])

r = [20, 12.5, 7.5, 5]
r_str = ['20', '12.5', '7.5', '5']


Reading galaxy data...
Filling in NaN values...
Copying galaxy coordinates...


In [2]:
ngal = {'20': [None] * len(coods), '12.5': [None] * len(coods), '7.5': [None] * len(coods), '5': [None] * len(coods)}
dgal = {'20': [None] * len(coods), '12.5': [None] * len(coods), '7.5': [None] * len(coods), '5': [None] * len(coods)}
max_fraction = {'20': [None] * len(coods), '12.5': [None] * len(coods), '7.5': [None] * len(coods), '5': [None] * len(coods)}
max_fraction_mass = {'20': [None] * len(coods), '12.5': [None] * len(coods), '7.5': [None] * len(coods), '5': [None] * len(coods)}
n_cluster_desc = {'20': [None] * len(coods), '12.5': [None] * len(coods), '7.5': [None] * len(coods), '5': [None] * len(coods)}



In [30]:
c = gals[10000:20000][['zn_x','zn_y','zn_z']].copy()

In [9]:
%timeit c.apply(lambda x: distance(x, gals[['zn_x','zn_y','zn_z']], dimensions), axis=1)

1 loops, best of 3: 11.8 s per loop


In [51]:
from periodic_kdtree import PeriodicCKDTree

In [52]:
T = PeriodicCKDTree(dimensions, gals[['zn_x','zn_y','zn_z']])

In [53]:
gal_index = T.query_ball_point(c, r=20)

In [54]:
gal_index.shape

(10000,)

In [99]:
%timeit gals.ix[gal_index[0]].groupby('z0_centralId', sort=False).mean()['z0_central_mcrit200']
agg_mvir = gals.ix[gal_index[0]].groupby('z0_centralId', sort=False).mean()['z0_central_mcrit200']

100 loops, best of 3: 2.81 ms per loop


In [98]:
%timeit np.unique(gals.ix[gal_index[0]]['z0_central_mcrit200'])

100 loops, best of 3: 1.25 ms per loop


In [214]:
%timeit Counter(gals.ix[gal_index[0]]['z0_central_mcrit200']).most_common()
c = Counter(gals.ix[gal_index[0]]['z0_central_mcrit200']).most_common()

100 loops, best of 3: 1.63 ms per loop


In [216]:
total = sum([x[1] for x in c])
%timeit sum([x[1] for x in c])

m_max = c[0][0]
cluster_descendants = [(x[0] > 1e4) for x in c]
%timeit [(x[0] > 1e4) for x in c]

n_cd = np.sum(cluster_descendants)
%timeit np.sum(cluster_descendants)

f_cd = float(np.sum([x[1] for x in c if (x[0] > 1e4)])) / total
%timeit float(np.sum([x[1] for x in c if (x[0] > 1e4)])) / total

max_frac = float(c[0][1]) / total
%timeit float(c[0][1]) / total

10000 loops, best of 3: 13.1 µs per loop
100000 loops, best of 3: 17.4 µs per loop
The slowest run took 4.41 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 17.2 µs per loop
10000 loops, best of 3: 22.6 µs per loop
The slowest run took 15.25 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 203 ns per loop


In [213]:
print total, m_max, n_cd, f_cd, max_frac

1475 144049.45 3 0.637288135593 0.456271186441


In [105]:
%timeit gals.ix[gal_index[0]].groupby('z0_centralId')['z0_centralId'].count()#.astype(float)
agg_count = gals.ix[gal_index[0]].groupby('z0_centralId')['z0_centralId'].count().astype(float)

100 loops, best of 3: 1.59 ms per loop


In [107]:
%timeit pd.DataFrame([agg_mvir, agg_count]).T
agg = pd.DataFrame([agg_mvir, agg_count]).T

100 loops, best of 3: 2.48 ms per loop


In [88]:
%timeit agg.loc[agg['z0_centralId'].idxmax()]['z0_central_mcrit200'] # find mass of most common descendant
m_max = agg.loc[agg['z0_centralId'].idxmax()]['z0_central_mcrit200'] # find mass of most common descendant

1000 loops, best of 3: 188 µs per loop


In [86]:
%timeit sum(agg_mvir > 1e4)
ncd = sum(agg_mvir > 1e4)

1000 loops, best of 3: 362 µs per loop


In [114]:
%timeit agg_count.max() / agg_count.sum()
max_frac = agg_count.max() / agg_count.sum()

10000 loops, best of 3: 111 µs per loop


In [129]:
from collections import Counter

In [130]:
gals.z0_centralId = gals.z0_centralId.astype(np.int64)

In [145]:
%timeit Counter(gals.loc[gal_index[0], 'z0_centralId']).most_common()

100 loops, best of 3: 1.86 ms per loop


In [ ]:
for i in range(len(gal_index)):

    n_galaxy = len(gal_index[i])
    ngal[R_str][j+i] = n_galaxy

    if n_galaxy == 0:
        m_max=0
        ncd=0
        agg_count=np.array([0])
        max_frac=0
    else:
        agg_mvir = gals.ix[gal_index[i]].groupby('z0_centralId').mean()['z0_central_mcrit200']
        agg_count = gals.ix[gal_index[i]].groupby('z0_centralId')['z0_centralId'].count().astype(float)

        agg = pd.DataFrame([agg_mvir, agg_count]).T
        
        m_max = agg.loc[agg['z0_centralId'].idxmax()]['z0_central_mcrit200'] # find mass of most common descendant
        ncd = sum(agg_mvir > 1e4)
        max_frac = agg_count.max() / agg_count.sum()

    max_fraction_mass[R_str][j+i] = m_max
    max_fraction[R_str][j+i] = max_frac
    n_cluster_desc[R_str][j+i] = ncd

Old loop for reference

In [ ]:
# can't calculate distances all in one go, so need to chunk
#for i,gals in z6_galaxies_mstar.groupby(np.arange(len(z6_galaxies_mstar))//n):
for i,c in coods.groupby(np.arange(len(coods))//n):

    if i % 5 == 0:
        print round(float(c.shape[0] * (i+1)) / coods.shape[0] * 100, 2), '%'

    # calculate distances
    dist = np.vstack(c.apply(lambda x: distance(x, gals[['zn_x','zn_y','zn_z']], dimensions), axis=1))

    for R, R_str in zip(r, r_str):

        gal_index = dist < R

        for i in range(len(gal_index)):

            n_galaxy = np.sum(gal_index[i])
            ngal[R_str].append(n_galaxy)

            if n_galaxy == 0:
                m_max=0
                ncd=0
                agg_count=np.array([0])
                max_frac=0
            else:
                agg_mvir = gals.ix[gal_index[i]].groupby('z0_centralId').mean()['z0_central_mcrit200']
                agg_count = gals.ix[gal_index[i]].groupby('z0_centralId')['z0_centralId'].count().astype(float)

                agg = pd.DataFrame([agg_mvir, agg_count]).T

                m_max = agg.loc[agg['z0_centralId'].idxmax()]['z0_central_mcrit200'] # find mass of most common descendant
                ncd = sum(agg_mvir > 1e4)
                max_frac = agg_count.max() / agg_count.sum()

            max_fraction_mass[R_str].append(m_max)
            max_fraction[R_str].append(max_frac)
            n_cluster_desc[R_str].append(ncd)